https://neo4j.com/labs/genai-ecosystem/graphrag-python/

https://neo4j.com/docs/neo4j-graphrag-python/current/user_guide_rag.html#

In [24]:
NEO4J_URI = "neo4j://localhost:7687"
NEO4J_USERNAME = "neo4j"
NEO4J_PASSWORD = "anatomy-exodus-pigment-colombo-octopus-1216"

In [25]:
# Neo4j Driver
import neo4j

neo4j_driver = neo4j.GraphDatabase.driver(
    NEO4J_URI,
    auth=(NEO4J_USERNAME, NEO4J_PASSWORD)
)

# LLM and Embedding Model
from neo4j_graphrag.llm import OpenAILLM
from neo4j_graphrag.embeddings.openai import OpenAIEmbeddings

llm=OpenAILLM(
   model_name="gpt-4o-mini",
   model_params={
       "temperature": 0 # turning temperature down for more deterministic results
   }
)

embedder = OpenAIEmbeddings()

In [46]:
q = "LG 유플러스에서 제공하는 이벤트"

In [47]:
# Vector Retriever
from neo4j_graphrag.generation import GraphRAG, RagTemplate
from neo4j_graphrag.retrievers import VectorRetriever

vector_retriever = VectorRetriever(
    neo4j_driver,
    index_name="text_embeddings",
    embedder=embedder,
    return_properties=["text"],
)

vector_rag = GraphRAG(llm=llm, retriever=vector_retriever, prompt_template=RagTemplate())

vector_rag.search(q, retriever_config={'top_k': 5}, return_context=True)

RagResultModel(answer="LG 유플러스에서 제공하는 이벤트는 다음과 같습니다:\n\n1. **달콤한 빈칸 맞추기 이벤트**: 정답을 맞추면 러쉬 기프트 세트, 황금올리브 콤보 세트, 배스킨라빈스 블록팩 등 다양한 경품을 받을 수 있습니다. [이벤트 참여하기](https://www.lguplus.com/benefit-event/ongoing/81669)\n\n2. **휴대폰 개통 고객 대상 이벤트**: 유플러스 닷컴에서 판매하는 휴대폰을 개통한 고객을 대상으로 다양한 경품을 제공합니다. 참여 방법은 개통 후 '이벤트 참여하기'를 통해 신청하면 됩니다. 당첨자 발표는 4월 23일입니다. 경품으로는 BESPOKE 큐브 공기청정기, 갤럭시 탭 S10 Ultra, 스타벅스 라떼 등 다양한 상품이 있습니다.\n\n3. **장기고객 및 유쓰 혜택**: 장기 고객과 유쓰 고객을 위한 특별 혜택이 제공됩니다. [자세히 보기](https://www.lguplus.com/mobile/device/phone)\n\n4. **진행 중 이벤트 및 출석체크 이벤트**: 다양한 이벤트가 진행 중이며, 참여할 수 있는 기회가 많습니다. [이벤트 목록 확인하기](https://www.lguplus.com/mobile/device/phone)\n\n이 외에도 다양한 혜택과 이벤트가 있으니 유플러스 공식 웹사이트를 방문해 보시기 바랍니다.", retriever_result=RetrieverResult(items=[RetrieverResultItem(content="{'text': '받아가세요](https://www.lguplus.com/static/pc-contents/images/prdv/20250307-013400-362-ap1w1SBx.png)](https://www.lguplus.com/benefit-event/ongoing/81683)\\n[ ![2025년 유플닷컴에선 혜택이 스르륵~ 달콤한 빈칸 맞추기 이벤트. 이벤트 정답 맞추고 경품 받아가세요. 러쉬 기프트 세트, 

In [49]:
# GraphRAG Vector Cypher Retriever
from neo4j_graphrag.generation import GraphRAG
from neo4j_graphrag.retrievers import VectorCypherRetriever

graph_retriever = VectorCypherRetriever(
    neo4j_driver,
    index_name="text_embeddings",
    embedder=embedder,
    retrieval_query="""
//1) Go out 2-3 hops in the entity graph and get relationships
WITH node AS chunk
MATCH (chunk)<-[:FROM_CHUNK]-(entity)-[relList:!FROM_CHUNK]-{1,2}(nb)
UNWIND relList AS rel

//2) collect relationships and text chunks
WITH collect(DISTINCT chunk) AS chunks, collect(DISTINCT rel) AS rels

//3) format and return context
RETURN apoc.text.join([c in chunks | c.text], '\n') +
  apoc.text.join([r in rels |
  startNode(r).name+' - '+type(r)+' '+r.details+' -> '+endNode(r).name],
  '\n') AS info
"""
)

graph_rag = GraphRAG(llm=llm, retriever=graph_retriever, prompt_template=RagTemplate())

graph_rag.search(q, retriever_config={'top_k': 5}, return_context=True)

RagResultModel(answer='LG 유플러스에서 제공하는 이벤트에는 선호번호 신청 이벤트와 당첨자 발표가 있습니다. 자세한 내용은 [여기](https://www.lguplus.com/mobile/device/phone)에서 확인할 수 있습니다.', retriever_result=RetrieverResult(items=[RetrieverResultItem(content="<Record info='   * [ 선호번호 신청 이벤트](https://www.lguplus.com/mobile/device/phone)\\n      * [ 당첨자 발표](https://www.lguplus.com/mobile/device/phone)\\n    * 유플닷컴 혜택\\n      * [ 닷컴 전용 혜택](https://www.lguplus.com/mobile/device/phone)\\n      * [ 닷컴 구매 혜택](https://www.lguplus.com/mobile/device/phone)\\n      * [ 포인트파크 할인](https://www.lguplus.com/mobile/device/phone)\\n    * 요금 할인 혜택\\n      * [ 모바일 요금 할인](https://www.lguplus.com/mobile/device/phone)\\n      * [ 선택약정 할인](https://www.lguplus.com/mobile/device/phone)\\n      * [ null\\nnull\\nnull\\nnull\\nnull\\nnull'>", metadata=None)], metadata={'__retriever': 'VectorCypherRetriever'}))

In [50]:
# Text2Cypher Retriever
from neo4j_graphrag.generation import GraphRAG
from neo4j_graphrag.retrievers import Text2CypherRetriever

text2cypher_retriever = Text2CypherRetriever(
    driver=neo4j_driver,
    llm=llm
)

text2cypher_rag = GraphRAG(llm=llm, retriever=text2cypher_retriever, prompt_template=RagTemplate())

text2cypher_rag.search(q, return_context=True)

RagResultModel(answer='LG 유플러스에서 제공하는 이벤트는 다양합니다. 예를 들어, 신규 가입자에게 제공되는 할인 혜택, 특정 요금제 가입 시 추가 데이터 제공, 또는 특정 기간 동안의 무료 체험 서비스 등이 있습니다. 또한, 시즌별로 진행되는 프로모션이나 경품 이벤트도 자주 열리니, LG 유플러스 공식 웹사이트나 앱을 통해 최신 정보를 확인하는 것이 좋습니다.', retriever_result=RetrieverResult(items=[], metadata={'cypher': "MATCH (o:Organization) WHERE o.name CONTAINS 'LG 유플러스' \nRETURN o", '__retriever': 'Text2CypherRetriever'}))